In [3]:
import sys
import os
import pandas as pd

path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)
    
sys.path

from primer3plus.utils import reverse_complement as rc
import primer3
from aqbt.contrib.uwbf import primer_utils
from aqbt import AquariumBuildTools
aqtools = AquariumBuildTools.from_toml('creds.secret.toml')
aqtools.sessions

aq = aqtools.sessions['production']['aquarium']

In [18]:
failed_plate_ids = [
    516595, 516600, 516597, 516599
]

success_plate_ids = [
    516613,
    516598,
    516596,
    516601,
]


def print_yeast(s):
    h1 = s.properties['Haploids'][0]
    h2 = s.properties['Haploids'][1]
    return "{id}: {name}\n  {h1id}: {h1}\n  {h2id}: {h2}".format(name=s.name, id=s.id, h1id=h1.id, h1=h1.name, h2id=h2.id, h2=h2.name)

print("-"*4 + " SUCCESS " + "-"*4)
for item_id in success_plate_ids:
    print(item_id)
    print(print_yeast(aq.Item.find(item_id).sample))
print()
print("-"*4 + " FAILED " + "-"*4)
for item_id in failed_plate_ids:
    print(item_id)
    print(print_yeast(aq.Item.find(item_id).sample))


---- SUCCESS ----
516613
36565: Dual Inducible NOR circuit (DSGRN topology, high performance CDM)
  36536: CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD-HO-KanMX-pRPL18B-rtTA-VP16 | HIS-HIS<7XTetO-pMinCyc1-URGR-W34> | URA-URA<pGRR-W8W34-URGR-W17> (v2) | ARS314-bleo<pGRR-W17W17-URGR-W10> (v2)
  36537: CEN.PK2 - MAT alpha ::: pMODKan-HO-pACT1-ZEV4 | pMOD8-pGALZ4-URGR-W8 | LEU-LEU<pGRR-W17W17-URGR-W20> (v2) | URA-URA<pGRR-W10W20-yeGFP> (v2)
516598
36569: Dual Inducible NOR circuit (GANDER topology, high performance CDM)
  36400: CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD-HO-KanMX-pRPL18B-rtTA-VP16 | HIS-HIS<7XTetO-pMinCyc1-URGR-W34>
  36382: CEN.PK2 - MAT alpha ::: pMODKan-HO-pACT1-ZEV4 | pMOD8-pGALZ4-URGR-W17 | URA-URA<pGRR-W17W34-yeGFP>
516596
36571: Dual Inducible OR circuit (GANDER topology, high performance CDM)
  36491: CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD-HO-KanMX-pRPL18B-rtTA-VP16 | HIS-HIS<7XTetO-pMinCyc1-URGR-W34> | URA-URA<pGRR-W8W34-URGR-W17> (v2)
  36540: CEN.PK2 - M

In [17]:
all_haploid_plates = []
for item in failed_plate_ids + success_plate_ids:
    for h in aq.Item.find(item).sample.properties['Haploids']:
        if 'gfp' in h.name.lower():
            plates = aq.Item.where({'sample_id': h.id, 'object_type_id': aq.ObjectType.find_by_name("Yeast Plate").id})
            plates = [p for p in plates if p.location != 'deleted']
            for p in plates:
                all_haploid_plates.append({'item_id': p.id, 'location': p.location, 'sample_id': p.sample_id})
                
                
import pandas as pd

pd.DataFrame(all_haploid_plates)


,item_id,location,sample_id
0,515919,DFP.6.2.20,36539
1,515502,DFP.6.1.8,36499
2,515920,DFP.6.3.0,36538
3,512121,DFP.6.0.9,36381
4,516589,DFP.6.3.9,36381
5,516439,DFP.6.3.8,36537
6,512393,DFP.6.0.8,36382
7,515921,DFP.6.3.4,36540
8,515508,DFP.6.1.13,36497


1. 516596 & 516595 should have reversed alpha strains.
2. 516598 has no GFP. Should rebuild GFP reporter in LEU or URA.
3. 516599 mating failed and has no GFP. Should rebuild GFP reporter in LEU or URA. Retransform with URA-URA<pGRR-W36W5-yeGFP> (v2). Question: does sample 36497 have GPF?
4. 516597 mating failed. Why? Eq. alpha strain "CEN.PK2 - MAT alpha ::: pMODKan-HO-pACT1-ZEV4 | pMOD8-pGALZ4-URGR-W5 | pMOD6-pGRR-W36W36-yeGFP + pGADT7-ADH1-iRGR-F7" was successful. Try that instead.
5. measure all haploids for prescence of GFP.
6. remake "CEN.PK2 - MAT alpha ::: pMODKan-HO-pACT1-ZEV4 | pMOD8-pGALZ4-URGR-W17 | URA-URA<pGRR-W17W34-yeGFP>" with URA-URA<pGRR-W17W34-yeGFP> (v2)
7. kill-curve for Bleo
    
I believe it appears any GFP reporter made by the BIOFAB does not work.

**TODO**

516613 and 516596 have mixed up haploid strains!!!!! WTF is wrong with you!

**516595, 36572: Dual Inducible OR circuit (GANDER topology, low performance CDM) (v2)**
wrong alpha strain was mated regardless. Could be a good negative control though.


**516600
36567: Dual Inducible OR circuit (DSGRN topology, high performance CDM)**
This may not have worked because bleo concentration was too high. The evidence for this is that plate 516601 had only baby colonies.

**516597
36570: Dual Inducible OR circuit (GANDER topology, low performance CDM)**
Clearly, somehow, the GFP is not in this circuit. Additionally, the LEU plasmid must have been dropped since the TRP-/LEU- selection did not work. Action 1: I think the solution here is to grow each haploid to saturation before mating. Action 2: create a LEU based GFP reporter and ensure this reporter is in transformation colony. Action 3: grow haploids in Trp- and Leu- respectively to ensure something did not go seriously seriously wrong here.

**516599
36568: Dual Inducible NOR circuit (GANDER topology, low performance CDM)**
    
Obviously, this has no trp selection (idiot). However, it also appears this has no GFP production and so the insert must have failed somehow in the same way that the other NOR gate has no GFP. Why is this happening?